In [ ]:
from utils import benchmark
import pandas as pd

embedding_models = ["pyannote/embedding", "models/embedding/student.ckpt"]
segmentation_models = ["pyannote/segmentation", "models/segmentation/student.ckpt"]
embedding_model_size = [4346366, 1510506]
segmentation_model_size = [1472749, 286570]

TRAIN = True

if TRAIN:
    results = []
    for embedding_model in embedding_models:
        for segmentation_model in segmentation_models:
            result = benchmark(embedding_model, segmentation_model)
            results.append(
                {
                    "embedding_model": (
                        "teacher" if "pyannote" in embedding_model else "student"
                    ),
                    "segmentation_model": (
                        "teacher" if "pyannote" in segmentation_model else "student"
                    ),
                    "diarization_error_rate": result["diarization error rate"][
                        "%"
                    ].TOTAL,
                    "false_alarm": result["false alarm"]["%"].TOTAL,
                    "percentage_of_teacher_size": (
                        embedding_model_size[embedding_models.index(embedding_model)]
                        + segmentation_model_size[
                            segmentation_models.index(segmentation_model)
                        ]
                    )
                    / (embedding_model_size[0] + segmentation_model_size[0])
                    * 100,
                }
            )

    df = pd.DataFrame(results)
    df.to_csv("results/evaluation.csv", index=False)
else:
    df = pd.read_csv("results/evaluation.csv")

print(df)

In [ ]:
# Convert to latex
latex = df.to_latex(
    index=False,
    caption="Evaluation of the teacher and student models on the AMI test dataset. Columns (from left to right): embedding model, segmentation model, diarization error rate, false alarms, parameter size of the pipeline as percentage of the original pipeline (\\textit{pyannote/embedding} with \\textit{pyannote/segmentation}). \\newline",
    label="tab:evaluation",
    header=[
        "Embedding model",
        "Segmentation model",
        "DER (\%)",
        "FA (\%)",
        "Size (\%)",
    ],
    float_format="{:0.2f}".format,
    column_format="llccc",
    escape=False,
)

# Necessary to change the font size to make the table fit
parts = latex.split("\label{tab:evaluation}")
parts_ = parts[1].split("\end{table}")
latex = (
    parts[0]
    + "\label{tab:evaluation}"
    + "{\\scriptsize "
    + parts_[0]
    + "}"
    + "\end{table}"
    + parts_[1]
)

with open("figures/evaluation.tex", "w") as f:
    f.write(latex)